In [68]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from libs.extract_hour import extract_hour

In [69]:
main_page = requests.get('https://comidadibuteco.com.br/butecos/belo-horizonte/')

html_main_page = BeautifulSoup(main_page.text, 'html.parser')

pagination = html_main_page.find_all('ul', class_='pagination')

In [70]:
index_1 = pagination[0].find_all('span')
index_2 = pagination[0].find_all('a')

index_list = index_1 + index_2

In [71]:
index_pages = []

for index in index_list:
    
    text = index.text

    if text.isdigit():

        index_pages.append(int(text))

num_pages = max(index_pages)
num_pages

11

In [72]:
current_page = requests.get(f'https://comidadibuteco.com.br/butecos/belo-horizonte/page/1/')
html_current_page = BeautifulSoup(current_page.text, 'html.parser')

links = html_current_page.find_all("a", string=lambda text: text and text.strip() == "Detalhes")

In [73]:
restaurants_links = []

for i in range(num_pages):
    
    i += 1
    current_page = requests.get(f'https://comidadibuteco.com.br/butecos/belo-horizonte/page/{i}/')
    html_current_page = BeautifulSoup(current_page.text, 'html.parser')

    link_DETALHES = html_current_page.find_all("a", string=lambda text: text and text.strip() == "Detalhes")

    for link in link_DETALHES:
        restaurants_links.append(link.get("href"))

len(restaurants_links)

125

In [74]:
main_columns = {
    "RESTAURANT_ID": [],
    "RESTAURANT_NAME": [],
    "MAIN_DISH": [],
    "MAIN_DISH_DESC": [],
    "STREET_NAME": [],
    "EST_NUMBER": [],
    "NEIGHBORHOOD": [],
    "CITY": [],
    "STATE": [],
    "COMPLEMENT": [],
    "PHONE_NUMBER": []
}

df_MAIN = pd.DataFrame(main_columns)
df_MAIN

,RESTAURANT_ID,RESTAURANT_NAME,MAIN_DISH,MAIN_DISH_DESC,STREET_NAME,EST_NUMBER,NEIGHBORHOOD,CITY,STATE,COMPLEMENT,PHONE_NUMBER


In [75]:
OH_columns = {
    "RESTAURANT_ID": [],
    "RESTAURANT_NAME": [],
    "WEEK_DAY": [],
    "OPENING_HOUR": [],
    "CLOSING_HOUR": []
}

df_OPENING_HOURS = pd.DataFrame(OH_columns)
df_OPENING_HOURS

,RESTAURANT_ID,RESTAURANT_NAME,WEEK_DAY,OPENING_HOUR,CLOSING_HOUR


In [76]:
req = requests.get('https://comidadibuteco.com.br/buteco/xico-do-churrasco/')
sp = BeautifulSoup(req.text, 'html.parser')

title = sp.find('h1', class_='section-title')
restaurant_name = title.text

section_text = sp.find('div', class_='section-text')
sections = section_text.find_all('p')

for section in sections:
    section_title = section.find('b').text
    print(section_title)
    print(section.text)


Xico Balaio
Xico Balaio Asinhas de frango com provolone com tempero.
Endereço: 
Endereço: Av Altamiro avelino Soares 920 – Castelo, Belo Horizonte – MG
Telefone: 
Telefone: 31993643195
Horario: 
Horario: Terça, das 17h à 0h | Quarta, das 17h à 0h | Quinta, das 17h à 0h | Sexta, das 17h à 0h | Sábado, das 17h à 0h | Domingo, das 11h às 18h |


In [77]:
count = 0

for link in restaurants_links:

    count += 1

    rest_id = f'CDB00{count}'

    restaurant_page = requests.get(link)
    html_restaurant_page = BeautifulSoup(restaurant_page.text, 'html.parser')

    title = html_restaurant_page.find('h1', class_='section-title')
    restaurant_name = title.text

    section_text = html_restaurant_page.find('div', class_='section-text')
    sections = section_text.find_all('p')

    address = None
    complement = None
    main_dish = None
    main_dish_desc = None
    opening_hours = None
    phone_number = None

    for section in sections:

        section_title = section.find('b').text

        match section_title:
            case 'Endereço: ':
                address = section.text.replace('Endereço: ', '').strip()
            case 'Complemento: ':
                complement = section.text.replace('Complemento: ', '').strip()
            case 'Telefone: ':
                phone_number = section.text.replace('Telefone: ', '').strip()
            case 'Horario: ':
                opening_hours = section.text.replace('Horario: ', '').strip()
            case _:
                main_dish = section_title.strip()
                main_dish_desc = section.text.replace(main_dish, '', 1).strip()

    try:
        if '|' in address:
            street_name = address.split('|')[0].split(',')[0].strip()
            est_number = address.split('|')[0].split(',')[1].strip()
            neighborhood = address.split('|')[1].split(',')[0].strip()
            city = address.split('|')[1].split(',')[1].split('–')[0].strip()
            state = address.split('|')[1].split(',')[1].split('–')[1].strip()
        else:
            street_name = address.split(',')[0].strip()
            est_number = address.split(',')[1].strip()
            neighborhood = None
            city = address.split(',')[2].split('–')[0].strip()
            state = address.split(',')[2].split('–')[1].strip()
    except:
        street_name = None
        est_number = None
        neighborhood = None
        city = None
        state = None

                
    new_element_MAIN = {
        "RESTAURANT_ID": rest_id,
        "RESTAURANT_NAME": restaurant_name,
        "MAIN_DISH": main_dish,
        "MAIN_DISH_DESC": main_dish_desc,
        "STREET_NAME": street_name,
        "EST_NUMBER": est_number,
        "NEIGHBORHOOD": neighborhood,
        "CITY": city,
        "STATE": state,
        "COMPLEMENT": complement,
        "PHONE_NUMBER": phone_number
    }

    df_MAIN.loc[len(df_MAIN)] = new_element_MAIN

    # HOURS
    hours_list = opening_hours.split('|')
    hours_list = [string for string in hours_list if string != '']

    for hour in hours_list:

        if 'SEGUNDA' in hour.upper():
            week_day = 'Segunda-Feira'
        elif 'TERÇA' in hour.upper():
            week_day = 'Terça-Feira'
        elif 'QUARTA' in hour.upper():
            week_day = 'Quarta-Feira'
        elif 'QUINTA' in hour.upper():
            week_day = 'Quinta-Feira'
        elif 'SEXTA' in hour.upper():
            week_day = 'Sexta-Feira'
        elif 'SÁBADO' in hour.upper() or 'SABADO' in hour.upper():
            week_day = 'Sábado'
        elif 'DOMINGO' in hour.upper():
            week_day = 'Domingo'
        else:
            print(restaurant_name)
            print(hour)
            raise('ERRO AO EXTRAIR DIA DA SEMANA!')
        
        try:
            opening_hour, closing_hour = extract_hour(hour)
        except:
            print(restaurant_name)
            print(hour)
            raise('ERRO AO EXTRAIR A HORA!')
        

        new_element_OH = {
            "RESTAURANT_ID": rest_id,
            "RESTAURANT_NAME": restaurant_name,
            "WEEK_DAY": week_day,
            "OPENING_HOUR": opening_hour,
            "CLOSING_HOUR": closing_hour
        }

        df_OPENING_HOURS.loc[len(df_OPENING_HOURS)] = new_element_OH

In [78]:
df_MAIN.to_excel('../data_bases/restaurants.xlsx', index=False, sheet_name='RESTAURANTS')

In [79]:
df_OPENING_HOURS.to_excel('../data_bases/opening_hours.xlsx', index=False, sheet_name='OPENING_HOURS')